### Exploratory Analysis of Stack Overflow Survey Data

#### Broadly speaking, we are interested in answering the following questions: 

1. What relates to remote vs. office work?
2. What relates to career satisfaction?
3. What relates to attitudes about diversity in the workplace?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import seaborn as sns
import os
import pprint
import column_rename_dicts as crd
%matplotlib inline
pd.options.display.max_seq_items = 500

In [2]:
#regex patterns needed in data cleaning
date_pattern = r"^([1-9]|1[012])[- /.]([1-9]|[12][0-9]|3[01])[- /.](19|20)\d\d$"

In [63]:
df_2017['DiversityImportant'].value_counts()

Agree                12011
Strongly agree        8022
Somewhat agree        7363
Disagree              2327
Strongly disagree     1180
Name: DiversityImportant, dtype: int64

In [3]:
#Load Data
df_2017 = pd.read_csv('./so-survey-2017/survey_results_public.csv')

In [4]:
df_2017[['DeveloperType','WebDeveloperType','MobileDeveloperType','NonDeveloperType']].head()
# pd.__version__

,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,Other,NaN,NaN,NaN
3,NaN,NaN,NaN,Data scientist
4,Mobile developer; Graphics programming; Deskto...,NaN,NaN,NaN


In [5]:
df_2017.groupby(['CompanyType']).size()

CompanyType
Government agency or public school/university                             2451
I don't know                                                              3233
I prefer not to answer                                                    1816
Non-profit/non-governmental organization or private school/university     1225
Pre-series A startup                                                      1288
Privately-held limited company, not in startup mode                      16709
Publicly-traded corporation                                               5871
Sole proprietorship or partnership, not in startup mode                   2831
Something else                                                             342
State-owned company                                                        670
Venture-funded startup                                                    2387
dtype: int64

The first few rows of the 2013 and 2014 data indicate that there are date values in the following variables that do not make sense, given the expected variable type:
* 'YearsProgram'
* 'CompanySize'
* 'NumDevsAtCompany'
* 'SizeOfTeam'

The fact that they are all 2013 dates suggests that they may be some sort of date stamp in lieu of a non-response. Since it is not clear what they ought to be, we will recode them as missing.

In [6]:
def recode_null_not_null_as_0_1(df, cols_not_to_recode):
    '''
    This function will split the data frame into columns that can easily be
    recoded as 0/1, and those that cannot. More specifically, if a column
    has a single non-NaN string value, and the relevant information is already contained
    in the column name, then it will convert that column to a 0/1 dummy
    '''
    
    # Split the dataframe into columns that will be operated on, and those that won't
    df_recode = df.drop(columns = cols_not_to_recode)
    df = df[cols_not_to_recode]
    df_recode = df_recode.notnull().astype('int')
    df = pd.concat([df, df_recode], axis=1)
    return df

In [ ]:
possible_vals = ["Take online courses", "Buy books and work through the exercises", 
                 "None of these", "Part-time/evening courses", "Return to college",
                 "Contribute to open source", "Conferences/meet-ups", "Bootcamp",
                 "Get a job as a QA tester", "Participate in online coding competitions",
                 "Master's degree", "Participate in hackathons", "Other"]

def clean_and_plot(df, title='Method of Educating Suggested', plot=True):
    '''
    INPUT 
        df - a dataframe holding the CousinEducation column
        title - string the title of your plot
        axis - axis object
        plot - bool providing whether or not you want a plot back
        
    OUTPUT
        study_df - a dataframe with the count of how many individuals
        Displays a plot of pretty things related to the CousinEducation column.
    '''
    study = df['CousinEducation'].value_counts().reset_index()
    study.rename(columns={'index': 'method', 'CousinEducation': 'count'}, inplace=True)
    study_df = t.total_count(study, 'method', 'count', possible_vals)

    study_df.set_index('method', inplace=True)
    if plot:
        (study_df/study_df.sum()).plot(kind='bar', legend=None);
        plt.title(title);
        plt.show()
    props_study_df = study_df/study_df.sum()
    return props_study_df
    
props_df = clean_and_plot(df)

In [31]:
def df_missingness_stats(df, year):
    df.name = year + ' data'
    print('Number of Rows in {}: {}'.format(df.name, df.shape[0]))
    print('Number of Columns in {}: {}'.format(df.name, df.shape[1]))
    print('Number of Columns in {} with no missing values: {}'.format(df.name, len(set(df.columns[~df.isnull().any()]))))
    print('Number of Columns in {} with > 75% missing values: {}'.format(df.name, 
                                                                         len(set(df.columns[df.isnull().sum()/len(df) > .75]))))
    print('Number of Columns in {} with all missing values: {}'.format(df.name, 
                                                                         len(set(df.columns[df.isnull().sum()/len(df) == 1]))))
    print('Columns in {} with no missing values: {}'.format(df.name, 
                                                            set(df.columns[~df.isnull().any()])))
    print('Columns in {} with > 75% missing values: {}'.format(df.name, 
                                                               set(df.columns[df.isnull().sum()/len(df) > .75])))

In [32]:
df_missingness_stats(df_2017, '2017')

Number of Rows in 2017 data: 51392
Number of Columns in 2017 data: 154
Number of Columns in 2017 data with no missing values: 7
Number of Columns in 2017 data with > 75% missing values: 14
Number of Columns in 2017 data with all missing values: 0
Columns in 2017 data with no missing values: {'ProgramHobby', 'Professional', 'EmploymentStatus', 'Respondent', 'FormalEducation', 'Country', 'University'}
Columns in 2017 data with > 75% missing values: {'ExCoder10Years', 'ExpectedSalary', 'ExCoderReturn', 'ExCoderWillNotCode', 'TimeAfterBootcamp', 'MobileDeveloperType', 'WebDeveloperType', 'NonDeveloperType', 'ExCoderNotForMe', 'ExCoderSkills', 'ExCoderBelonged', 'ExCoderBalance', 'YearsCodedJobPast', 'ExCoderActive'}


In [33]:
df_2017['Professional'].value_counts()

Professional developer                                  36131
Student                                                  8224
Professional non-developer who sometimes writes code     5140
Used to be a professional developer                       983
None of these                                             914
Name: Professional, dtype: int64

In [34]:
df_2017['ProgramHobby'].value_counts()

Yes, I program as a hobby                    24801
Yes, both                                    13756
No                                            9787
Yes, I contribute to open source projects     3048
Name: ProgramHobby, dtype: int64

In [35]:
df_2017['University'].value_counts()

No                     37543
Yes, full-time          9369
Yes, part-time          3352
I prefer not to say     1128
Name: University, dtype: int64

In [36]:
df_2017['EmploymentStatus'].value_counts()

Employed full-time                                      36148
Independent contractor, freelancer, or self-employed     5233
Employed part-time                                       3180
Not employed, and not looking for work                   2791
Not employed, but looking for work                       2786
I prefer not to say                                      1086
Retired                                                   168
Name: EmploymentStatus, dtype: int64

In [37]:
df_2017['FormalEducation'].value_counts()

Bachelor's degree                                                    21609
Master's degree                                                      11141
Some college/university study without earning a bachelor's degree     8129
Secondary school                                                      5908
Doctoral degree                                                       1308
I prefer not to answer                                                1109
Primary/elementary school                                             1047
Professional degree                                                    715
I never completed any formal education                                 426
Name: FormalEducation, dtype: int64

In [38]:
df_2017['MajorUndergrad'].value_counts()

Computer science or software engineering                        21409
Computer engineering or electrical/electronics engineering       4364
Computer programming or Web development                          3883
Information technology, networking, or system administration     2140
A natural science                                                1865
A non-computer-focused engineering discipline                    1789
Mathematics or statistics                                        1648
Something else                                                   1060
A humanities discipline                                           901
A business discipline                                             900
Management information systems                                    654
Fine arts or performing arts                                      646
A social science                                                  633
I never declared a major                                          592
Psychology          

In [39]:
df_2017['HomeRemote'].value_counts()

A few days each month                                      15454
Never                                                      13975
All or almost all the time (I'm full-time remote)           4905
Less than half the time, but at least one day each week     4147
More than half, but not all, the time                       1909
It's complicated                                            1849
About half the time                                         1769
Name: HomeRemote, dtype: int64

In [40]:
df_2017['CompanySize'].value_counts()

20 to 99 employees          8587
100 to 499 employees        7274
10,000 or more employees    5680
10 to 19 employees          4103
1,000 to 4,999 employees    3831
Fewer than 10 employees     3807
500 to 999 employees        2486
5,000 to 9,999 employees    1604
I don't know                 869
I prefer not to answer       681
Name: CompanySize, dtype: int64

In [55]:
df_2017['CompanyType'].value_counts()

Privately-held limited company, not in startup mode                      16709
Publicly-traded corporation                                               5871
I don't know                                                              3233
Sole proprietorship or partnership, not in startup mode                   2831
Government agency or public school/university                             2451
Venture-funded startup                                                    2387
I prefer not to answer                                                    1816
Pre-series A startup                                                      1288
Non-profit/non-governmental organization or private school/university     1225
State-owned company                                                        670
Something else                                                             342
Name: CompanyType, dtype: int64

In [56]:
df_2017['YearsProgram'].value_counts()

20 or more years    8817
4 to 5 years        3865
3 to 4 years        3701
5 to 6 years        3572
2 to 3 years        3251
9 to 10 years       3197
6 to 7 years        2869
1 to 2 years        2773
7 to 8 years        2466
10 to 11 years      2178
14 to 15 years      2011
8 to 9 years        1910
15 to 16 years      1669
Less than a year    1484
11 to 12 years      1389
12 to 13 years      1307
13 to 14 years      1092
16 to 17 years      1046
19 to 20 years      1021
17 to 18 years       879
18 to 19 years       648
Name: YearsProgram, dtype: int64

In [57]:
df_2017['YearsCodedJob'].value_counts()

1 to 2 years        5289
2 to 3 years        4767
3 to 4 years        4003
4 to 5 years        3405
20 or more years    3067
Less than a year    3034
5 to 6 years        2990
9 to 10 years       1956
6 to 7 years        1912
10 to 11 years      1678
7 to 8 years        1638
8 to 9 years        1287
15 to 16 years       845
14 to 15 years       840
11 to 12 years       838
12 to 13 years       741
16 to 17 years       694
17 to 18 years       545
13 to 14 years       531
19 to 20 years       427
18 to 19 years       403
Name: YearsCodedJob, dtype: int64

In [58]:
df_2017['YearsCodedJobPast'].value_counts()

20 or more years    154
1 to 2 years         80
2 to 3 years         78
4 to 5 years         66
3 to 4 years         65
9 to 10 years        59
Less than a year     57
10 to 11 years       50
5 to 6 years         48
7 to 8 years         46
15 to 16 years       42
14 to 15 years       39
6 to 7 years         30
11 to 12 years       28
8 to 9 years         28
19 to 20 years       23
12 to 13 years       20
16 to 17 years       19
17 to 18 years       19
13 to 14 years       17
18 to 19 years        6
Name: YearsCodedJobPast, dtype: int64

In [61]:
df_2017['DeveloperType'].value_counts()

Web developer                                                                                                                                                                                                                                                                       10683
Web developer; Desktop applications developer                                                                                                                                                                                                                                        1849
Mobile developer                                                                                                                                                                                                                                                                     1556
Web developer; Mobile developer                                                                                                                           

In [41]:
def create_dummy_df(df, cols_to_dummy, dummy_na=True):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    for col in cols_to_dummy:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], 
                                                                 prefix=col, 
                                                                 prefix_sep='_', 
                                                                 drop_first=True,
                                                                 dummy_na=dummy_na
                                                                )], axis=1)
        except:
            continue

    return df

In [42]:
#creating labelEncoder
def process_y_var_split_data(df, yvar):
    df[yvar] = df[yvar].str.replace('<', 'less than ')
    df[yvar] = df[yvar].str.replace('>', 'greater than ')
    df[yvar] = df[yvar].astype('str')
    df = df[df[yvar]!='nan']
    
    print(df[yvar].value_counts())
    le = preprocessing.LabelEncoder()
    X = df.drop(columns=[yvar])
    y = le.fit_transform(df[yvar])
    
    return X, y 


In [43]:
X_2013, y_2013 = process_y_var_split_data(df_2013, 'TotalCompensation')

NameError: name 'df_2013' is not defined

In [44]:
X_2014, y_2014 = process_y_var_split_data(df_2014, 'TotalCompensation')

NameError: name 'df_2014' is not defined

In [45]:
X_2015, y_2015 = process_y_var_split_data(df_2015, 'Compensation')

NameError: name 'df_2015' is not defined

In [46]:
# Split dataset into training set and test set
def compare_classifiers(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42) # 70% training and 30% test


    #Create a Gaussian Classifier
    gnb = GaussianNB()

    #Train the model using the training sets
    gnb.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_gnb = gnb.predict(X_test)

    # Logistic Regression Classifier
    logreg = LogisticRegression()

    #Train the model using the training sets
    logreg.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_lr = logreg.predict(X_test)

    # Random Forest Classifier
    rf = RandomForestClassifier()

    #Train the model using the training sets
    rf.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_rf = rf.predict(X_test)

    # Adaboost Classifier
    ada = AdaBoostClassifier()

    #Train the model using the training sets
    ada.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_ada = ada.predict(X_test)

    print("GNB Accuracy:",metrics.accuracy_score(y_test, y_pred_gnb))

    print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, y_pred_lr))

    print("Random Forest Accuracy:",metrics.accuracy_score(y_test, y_pred_rf))

    print("AdaBoost Accuracy:",metrics.accuracy_score(y_test, y_pred_ada))


In [47]:
# Split dataset into training set and test set
def compare_classifiers(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42) # 70% training and 30% test

    #Create a Gaussian Classifier
    gnb = GaussianNB()

    #Train the model using the training sets
    gnb.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_gnb = gnb.predict(X_test)

    # Logistic Regression Classifier
    logreg = LogisticRegression()

    #Train the model using the training sets
    logreg.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_lr = logreg.predict(X_test)

    # Random Forest Classifier
    rf = RandomForestClassifier()

    #Train the model using the training sets
    rf.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_rf = rf.predict(X_test)

    # Adaboost Classifier
    ada = AdaBoostClassifier()

    #Train the model using the training sets
    ada.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred_ada = ada.predict(X_test)

    print("GNB Accuracy:",metrics.accuracy_score(y_test, y_pred_gnb))

    print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, y_pred_lr))

    print("Random Forest Accuracy:",metrics.accuracy_score(y_test, y_pred_rf))

    print("AdaBoost Accuracy:",metrics.accuracy_score(y_test, y_pred_ada))

In [48]:
compare_classifiers(X_2014, y_2014)

NameError: name 'X_2014' is not defined

In [49]:
compare_classifiers(X_2015, y_2015)

NameError: name 'X_2015' is not defined

In [50]:
# pp.pprint(crd.column_name_map_2014)

In [51]:
# pp.pprint(list(df_2014.columns))

In [52]:
def plot_value_counts(df, col, plot_title):
    status_vals = df[col].value_counts() 
    print(status_vals)
    # The below is a bar chart of the proportion of observations in each category of df[col]
    (status_vals/df.shape[0]).plot(kind="bar");
    plt.title("What kind of developer are you?");

In [53]:
def get_description(column_name, schema=schema):
    '''
    INPUT - schema - pandas dataframe with the schema of the developers survey
            column_name - string - the name of the column you would like to know about
    OUTPUT - 
            desc - string - the description of the column
    '''
    desc = schema['Question'][schema['Column']==column_name].values[0]
    return desc

NameError: name 'schema' is not defined

In [54]:
possible_vals = ["Take online courses", "Buy books and work through the exercises", 
                 "None of these", "Part-time/evening courses", "Return to college",
                 "Contribute to open source", "Conferences/meet-ups", "Bootcamp",
                 "Get a job as a QA tester", "Participate in online coding competitions",
                 "Master's degree", "Participate in hackathons", "Other"]

def clean_and_plot(df, title='Method of Educating Suggested', plot=True):
    '''
    INPUT 
        df - a dataframe holding the CousinEducation column
        title - string the title of your plot
        axis - axis object
        plot - bool providing whether or not you want a plot back
        
    OUTPUT
        study_df - a dataframe with the count of how many individuals
        Displays a plot of pretty things related to the CousinEducation column.
    '''
    study = df['CousinEducation'].value_counts().reset_index()
    study.rename(columns={'index': 'method', 'CousinEducation': 'count'}, inplace=True)
    study_df = t.total_count(study, 'method', 'count', possible_vals)

    study_df.set_index('method', inplace=True)
    if plot:
        (study_df/study_df.sum()).plot(kind='bar', legend=None);
        plt.title(title);
        plt.show()
    props_study_df = study_df/study_df.sum()
    return props_study_df
    
props_df = clean_and_plot(df)

NameError: name 'df' is not defined